# Introduction

As a business owner, customer reviews can be a valuable source of insight. Imagine being able to gradually monitor areas for improvement that increase customer satisfaction and highlight the best parts of the business for effective branding.

This project aims to segment user reviews into several topics for easier analysis.

The key components of our project include:
- **Review clustering**: to segment customer reviews into distinct clusters by representing the reviews as word embedding (combination of pre-trained LLM and self-train model),
- **Sentiment analysis**: to classify the sentiment of a review as positive or negative,
- **Topic labeling**: to label review topics within each cluster using a large language model (LLM).


## Dataset

The dataset for this project is [Google Local dataset](https://cseweb.ucsd.edu/~jmcauley/datasets.html#google_local) obtained from J. McAuley lab.

Originally, the dataset contains millions of business reviews from across the United States up to 2021. However, for the sake of simplicity and due to limited resources for this project, we focus exclusively on one state and one business type: **tourist attractions in Hawaii**.

In the end, we limit to 1000 locations with approximately 144k reviews.

In [ ]:
!pip install -q en_core_web_sm transformers sentence-transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
from google.cloud import storage
from datetime import datetime

from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import openai
import io

In [ ]:
def download_csv_from_gcs(bucket, file_name,
                          date_columns=None, col_names=None):
    """ A function to download dataset from GCS. """

    blob = bucket.blob(file_name)
    data = blob.download_as_text()
    df = pd.read_csv(io.StringIO(data),
                     parse_dates=date_columns,
                     usecols=col_names)
    return df

In [ ]:
# Create a client GCS and get the specified bucket
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)

In [ ]:
# Download the dataset from GCS
reviews_df = download_csv_from_gcs(bucket, REVIEW_CSV)

In [ ]:
reviews_df.head()

,business_id,user_id,time,text
0,0x7c006afc71065bd1:0x7a706dc72f4623ee,113728016128003691063,2021-08-31 04:41:40.565,We needed sun glasses before a boat ride and n...
1,0x7c006afc71065bd1:0x7a706dc72f4623ee,101488063088102775913,2021-08-24 07:48:29.263,Reasonable Prices Has quick food to go
2,0x7c006afc71065bd1:0x7a706dc72f4623ee,100536389119109882523,2021-07-25 00:58:57.672,Convenient liquor store with a little of every...
3,0x7c006afc71065bd1:0x7a706dc72f4623ee,110229694025406635705,2021-07-24 19:28:41.268,I love the spam musubis It's the bomb
4,0x7c006afc71065bd1:0x7a706dc72f4623ee,108697060494095206753,2021-04-08 21:00:29.786,Had a good selection of items at fairly decent...


In [ ]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144762 entries, 0 to 144761
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  144762 non-null  object
 1   user_id      144762 non-null  object
 2   time         144762 non-null  object
 3   text         144762 non-null  object
dtypes: object(4)
memory usage: 4.4+ MB


In [ ]:
reviews_df["time"] = pd.to_datetime(reviews_df["time"])

In [ ]:
# Count the number of unique businesses and users
reviews_df[["business_id", "user_id"]].nunique()

,0
business_id,1035
user_id,72582


In [ ]:
# Count the average number of reviews for each business
int(reviews_df.groupby("business_id")["text"].count().mean())

139

In [ ]:
# Check the missing values
reviews_df.isna().sum()

,0
business_id,0
user_id,0
time,0
text,0


# Dataset Preparation

In [ ]:
from transformers import pipeline

import en_core_web_sm
import re

In [ ]:
spacy_nlp = en_core_web_sm.load()

In [ ]:
class ProcessDataset():
    """
      A class for preprocessing Reviews data for training downstream models.
      Preprocessing includes:
        - clean, split, and expand setences,
        - tokenize, lemmatize, and remove stop words from sentences
    """

    def __init__(self, spacy_nlp):

        self.nlp = spacy_nlp

    def _clean_text(self, text):
        """ Clean text from unnecessary tokens/substrings """

        # Remove emoji patterns
        emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"  # Emoticons
            "\U0001F300-\U0001F5FF"  # Symbols & pictographs
            "\U0001F680-\U0001F6FF"  # Transport & map symbols
            "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
            "\U00002700-\U000027BF"  # Dingbats
            "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
            "\U00002600-\U000026FF"  # Misc symbols
            "\U00002B50-\U00002B59"  # Stars
            "]+", flags=re.UNICODE
        )
        text = emoji_pattern.sub(r"", text)

        # Extracts text between '(Translated by Google)' and '(Original)'.
        match = re.search(r"\(Translated by Google\)(.+?)  ", text)
        if match:
            text = match.group(1)

        return text

    def _split_and_tokenize(self, text):
        """
          Splits text into sentences using the spaCy model.
          Also tokenize and lemmatize.
        """

        sents = [sent for sent in self.nlp(text.lower()).sents if sent.text]

        full_sents = [sent.text for sent in sents]

        tokenized = [[ token.lemma_ for token in sent
                      if token.is_alpha and not token.is_punct ]
                     for sent in sents ]
        tokenized = [" ".join(sent) for sent in tokenized]
        # We do the above operation so that it can be exploded later

        return full_sents, tokenized

    def transform(self, dataset):
        """ The main text processing function. """

        data = dataset.copy()

        # Clean, split, and expand sentences
        data["text"] = data["text"].apply(self._clean_text)
        data.loc[:, ["processed_text", "tokens"]] = data["text"].apply(
            self._split_and_tokenize).apply(
                lambda x: pd.Series(x, index=["processed_text", "tokens"]))

        data = data.explode(["processed_text", "tokens"]).reset_index(drop=True)
        data = data[data["processed_text"].str.len() >= 10]

        # Tokenize sentence
        data["tokens"] = data["tokens"].apply(lambda x: x.split())
        data = data[data["tokens"].apply(lambda x: len(x) >= 2)]

        return data

In [ ]:
data_processor = ProcessDataset(spacy_nlp)

In [ ]:
processed_dataset = data_processor.transform(reviews_df)

In [ ]:
# NOTE: we use `processed_text` as the transformer input and `tokens` for w2v
processed_dataset[["processed_text", "tokens"]].head()

,processed_text,tokens
0,we needed sun glasses before a boat ride and n...,"[we, need, sun, glass, before, a, boat, ride, ..."
1,this little mom and pop convenience store had ...,"[this, little, mom, and, pop, convenience, sto..."
2,we both got our sunglasses for a great price a...,"[we, both, get, our, sunglass, for, a, great, ..."
3,reasonable prices has quick food to go,"[reasonable, price, have, quick, food, to, go]"
4,convenient liquor store with a little of every...,"[convenient, liquor, store, with, a, little, o..."


In [ ]:
processed_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 323490 entries, 0 to 340746
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   business_id     323490 non-null  object        
 1   user_id         323490 non-null  object        
 2   time            323490 non-null  datetime64[ns]
 3   text            323490 non-null  object        
 4   processed_text  323490 non-null  object        
 5   tokens          323490 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 17.3+ MB


In [ ]:
cols = ["processed_text", "tokens"]
processed_dataset[cols].to_parquet("processed_dataset_ckpt.parquet",
                                   index=False)
!gsutil cp "processed_dataset_ckpt.parquet" "gs://customer_review_hawaii/data/"

Copying file://processed_dataset_ckpt.parquet [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/16.5 MiB.                                     


# `SentenceTransformer` Model

Our goal in this project is to group customer reviews into clusters based on the topics of the reviews. To do this, the appropriate feature representation is word vectors.

To obtain word vectors from a sentence, we commonly use deep neural networks, either by training a model ourself or by using a pretrained one. Nowadays, several pretrained models are available for extracting word vectors.

The benefit of using a pretrained model is that its trained using advanced network on a large and diverse dataset. Though, the drawback is that it may be less specific to the dataset. However, in general, these pretrained models perform well in most cases.

Hence, we use the `SentenceTransformer` model obtained from [this project](https://github.com/UKPLab/sentence-transformers). To be specific, we use the `all-MiniLM-L6-v2` model.

## Clustering

ADD PIC HERE!

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_llm = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
model_sentiment = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [351]:
# Helper functions for data selection, cluster evaluation, and visualization

def select_dataset(dataset, business_id, model_sentiment):
    """
        Select dataset based on the business_id,
        also encode the sentiment using a pretrained model.
    """

    # Select data based on `business_id`
    data = dataset[dataset["business_id"] == business_id].copy()

    # Select reviews from the last six months
    data["time"] = pd.to_datetime(data["time"])
    time_limit = data["time"].max() - pd.DateOffset(months=6)
    data = data[data["time"] >= time_limit]

    # Extract the sentiment using a pretrained model
    data["sentiment"] = data["processed_text"].apply(lambda x: model_sentiment(x))
    data["sentiment"] = data["sentiment"].apply(lambda x: x[0]["label"])

    return data

def evaluate_cluster(X, y, name):
    """
        Evaluate clustering result using 3 evaluation metrics:
        1. `silhouette_score`: a metric used to calculate the goodness of fit
            of a clustering algorithm. Its value ranges from -1 to 1.
        2. `davies_bouldin_score`: the average similarity measure of each cluster
            with its most similar cluster. The minimum value is 0 for better model.
        3. `calinski_harabasz_score`: the ratio of the sum of between-cluster dispersion
            and of within-cluster dispersion. Higher index indicates separable clusters.
    """
    evaluation_scores = {}
    evaluation_scores["silhouette_score"] = [silhouette_score(X, y)]
    evaluation_scores["davies_bouldin_score"] = [davies_bouldin_score(X, y)]
    evaluation_scores["calinski_harabasz_score"] = [calinski_harabasz_score(X, y)]

    return pd.DataFrame(evaluation_scores, index=[f"{name.title()} Reviews Clustering"]).T

def visualize_cluster(fig, df, x_column, y_column, col=None, sentiment="positive"):

    # Perform PCA transformation for visualization
    pca = PCA(n_components=2)
    df["w2v_pca"] = list(pca.fit_transform(np.vstack(df[x_column].values)))
    df[["w2v_pca_x", "w2v_pca_y"]] = pd.DataFrame(df["w2v_pca"].tolist(), index=df.index)

    df[y_column] = df[y_column].astype("category")

    # Plot with px.scatter using the new PCA columns
    fig_px = px.scatter(df,
                        x="w2v_pca_x",
                        y="w2v_pca_y",
                        color=df[y_column],
                        hover_data={"processed_text": True},
                        color_discrete_sequence=px.colors.qualitative.Set1)

    fig_px.update_traces(showlegend=False)

    # Add this plot to the subplot column
    for trace in fig_px.data:
        fig.add_trace(trace, row=1, col=col)

In [218]:
class HDBSCANClustering(BaseEstimator, TransformerMixin):

    def __init__(self, apply_pca=False):
        self.model = HDBSCAN(metric="cosine",
                             min_cluster_size=2,
                             min_samples=2)
        self.labels_ = None

        self.apply_pca = apply_pca
        self.pca = PCA()

    def fit(self, X, y=None):
        """ Fit the cluster. """

        if self.apply_pca:
            # Perform PCA transformation before clustering
            X = self.pca.fit_transform(X)

        self.labels_ = self.model.fit_predict(X)
        return self

    def transform(self, X):
        """ Return the label """

        return self.labels_

In [219]:
class LLMEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, model):
        # Load the Transformer model
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """ Transform the data using the transformer model. """

        wv = X.apply(self.model.encode)
        return wv

In [ ]:
llm_encoder = LLMEncoder(model_llm)

In [338]:
business_id = np.random.choice(processed_dataset["business_id"])

In [352]:
selected_data = select_dataset(processed_dataset, business_id, model_sentiment)

# Get the word vector from the pretrained transformer model
selected_data["llm"] = llm_encoder.transform(selected_data["processed_text"])

# Initialize evaluation results
evaluation = []

# Initialize subplots
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Positive Reviews", "Negative Reviews"])

for i, sentiment in enumerate(["positive", "negative"], start=1):

    # Clustering
    clust = HDBSCANClustering(apply_pca=True)
    X = selected_data[selected_data["sentiment"] == sentiment].copy()
    X_ = np.vstack(X["llm"].values)
    X["llm_label"] = clust.fit_transform(X_)

    # Evaluation
    evaluation.append(evaluate_cluster(X_, X["llm_label"], sentiment))

    # Visualization
    visualize_cluster(fig, X, "llm", "llm_label",
                      col=i, sentiment=sentiment)

fig.update_layout(
    title="Customer Review Clusters using LLM Word Vector"
)
fig.show()

In [353]:
# Evaluation
pd.concat(evaluation, axis=1)

,Positive Reviews Clustering,Negative Reviews Clustering
silhouette_score,0.022505,0.147159
davies_bouldin_score,1.826998,1.346603
calinski_harabasz_score,1.760827,2.191600


# Word2Vec Model

In the previous section, we saw that clustering with word vectors provides some separation, but the results are still insufficient.

A common approach to improve the fit of a pretrained model to a specific dataset is fine-tuning. However, this requires large computational resources.

Alternatively, we can train a custom model and then *concatenate* its word vectors with those from the pretrained model before clustering. For this project, we chose this approach. We trained a custom skip-gram Word2Vec model on our review dataset to create word vectors specific to our data.

## Training

In [ ]:
from gensim.models import Word2Vec
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import HDBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [189]:
class W2VEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, model=None, normalize=True):
        # Use any existing w2v model
        self.model = model
        self.normalize = normalize

    def fit(self, X, y=None):
        """ Train the w2v model. """

        if not self.model:
            self.model = Word2Vec(X, vector_size=384,
                                  window=3, min_count=1,
                                  negative=10, sg=1,
                                  compute_loss=True, epochs=10,
                                  alpha=0.001, min_alpha=0.0001)
        print("Finished training!")
        print(f"Latest training loss (cumulative): {self.model.get_latest_training_loss()}")

        return self

    def transform(self, X):
        """ Transform the data using the learned w2v model. """

        wv = X.apply(lambda tokens: [self.model.wv[token] for token in tokens
                                     if token in self.model.wv])
        if self.normalize:
            # Normalize the vector
            wv = wv.apply(lambda v: v / np.linalg.norm(v))
        wv = wv.apply(lambda v: np.array(v).mean(axis=0))
        return wv

In [190]:
# Train the Word2Vec model
w2v_encoder = W2VEncoder()
w2v_encoder.fit(processed_dataset["tokens"])

Finished training!
Latest training loss (cumulative): 56233140.0


W2VEncoder(model=<gensim.models.word2vec.Word2Vec object at 0x793f4027fbe0>)

In [191]:
# Save model for future access
# w2v_encoder.model.save(f"model_w2v")

## Clustering

In [354]:
selected_data = select_dataset(processed_dataset, business_id, model_sentiment)

# Get the word vector from the trained Word2Vec model
selected_data["w2v"] = w2v_encoder.transform(selected_data["tokens"])

# Initialize evaluation results
evaluation = []

# Initialize subplots
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Positive Reviews", "Negative Reviews"])

for i, sentiment in enumerate(["positive", "negative"], start=1):

    # Clustering
    clust = HDBSCANClustering(apply_pca=True)
    X = selected_data[selected_data["sentiment"] == sentiment].copy()
    X_ = np.vstack(X["w2v"].values)
    X["w2v_label"] = clust.fit_transform(X_)

    # Evaluation
    evaluation.append(evaluate_cluster(X_, X["w2v_label"], sentiment))

    # Visualization
    visualize_cluster(fig, X, "w2v", "w2v_label",
                      col=i, sentiment=sentiment)

fig.update_layout(title="Customer Review Clusters - Positive and Negative")
fig.show()

In [355]:
# Evaluation
pd.concat(evaluation, axis=1)

,Positive Reviews Clustering,Negative Reviews Clustering
silhouette_score,0.166748,0.382662
davies_bouldin_score,1.492249,0.842067
calinski_harabasz_score,9.603434,8.119819


# Concatenated Word Vectors

In this section, we **concatenate** the Word2Vec embedding with the one generated by `SentenceTransformers`.

We also train an autoencoder model to **reduce the dimensionality** of our combined embeddings, as in theory it can effectively handles complex, non-linear relationships that traditional linear methods like PCA may not fully capture.

Before training the autoencoder, we apply PCA to determine the optimal dimensionality required to retain sufficient information from the embeddings.

In [356]:
selected_data = select_dataset(processed_dataset, business_id, model_sentiment)

# Get the word vector from the trained Word2Vec model
selected_data["w2v"] = w2v_encoder.transform(selected_data["tokens"])
selected_data["llm"] = llm_encoder.transform(selected_data["processed_text"])
selected_data["mix"] = selected_data[["w2v", "llm"]].apply(
                          lambda x: np.hstack([x["w2v"], x["llm"]]), axis=1
                      )

# Initialize evaluation results
evaluation = []

# Initialize subplots
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Positive Reviews", "Negative Reviews"])

for i, sentiment in enumerate(["positive", "negative"], start=1):

    # Clustering
    clust = HDBSCANClustering(apply_pca=True)
    X = selected_data[selected_data["sentiment"] == sentiment].copy()
    X_ = np.vstack(X["mix"].values)
    X["mix_label"] = clust.fit_transform(X_)

    # Evaluation
    evaluation.append(evaluate_cluster(X_, X["mix_label"], sentiment))

    # Visualization
    visualize_cluster(fig, X, "mix", "mix_label",
                      col=i, sentiment=sentiment)

fig.update_layout(title="Customer Review Clusters - Positive and Negative")
fig.show()

In [357]:
# Evaluation
pd.concat(evaluation, axis=1)

,Positive Reviews Clustering,Negative Reviews Clustering
silhouette_score,0.007298,0.146647
davies_bouldin_score,1.966938,1.347486
calinski_harabasz_score,1.620542,2.192180


## Determine the size of the latent vars

We estimate the size of the latent variables using PCA.

However, due to the size of our dataset, we only use 10% of it for this analysis.

We found that a 410-dimensions vector is sufficient to capture 99% of the total variance.

In [245]:
sample_10p = processed_dataset.sample(int(0.1 * len(processed_dataset)))
sample_10p["w2v"] = w2v_encoder.transform(sample_10p["tokens"])
sample_10p["llm"] = llm_encoder.transform(sample_10p["processed_text"])
sample_10p["mix"] = sample_10p[["w2v", "llm"]].apply(
                          lambda x: np.hstack([x["w2v"], x["llm"]]), axis=1
                      )

In [369]:
pca = PCA()
X = np.vstack(sample_10p["mix"].values)
pca.fit(X)

cumulative_var = np.cumsum(pca.explained_variance_ratio_)

threshold = 0.999
n = np.argmax(cumulative_var >= threshold) + 1
print(n)

406


We can see from the plot above that the number to get at least 95% variance from the original data need around 80 dimension.

With this information, we will train autoencoder accordingly.

In [ ]:
# class PCAEncoder(BaseEstimator, TransformerMixin):
#     def __init__(self, n):
#         self.pca = PCA(n_components=n)

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         encoded = self.pca.fit_transform(batch)

#         return encoded

## Dataset preparation

In [249]:
from sklearn.decomposition import PCA
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

In [290]:
# Turn the DataFrame to Pytorch Dataset

class AutoEncoderDataset(Dataset):
    def __init__(self, dataset, w2v_encoder, llm_encoder, source="mix"):
        self.data = dataset.copy()

        self.w2v_encoder = w2v_encoder
        self.llm_encoder = llm_encoder

        assert source in ["w2v", "llm", "mix"]
        self.source = source

    def get_feature_dim(self):
        """ Return the feature (word vector) dimension """

        return len(self.__getitem__(0))

    def __len__(self):
        """ Return the length of the dataset """

        return len(self.data)

    def __getitem__(self, idx):
        """ Return row data """

        data = self.data.iloc[idx, :].copy()

        data["w2v"] = self.w2v_encoder.transform(data[["tokens"]]).iloc[0]
        data["llm"] = self.llm_encoder.transform(data[["processed_text"]]).iloc[0]
        data["mix"] = np.hstack(data[["w2v", "llm"]].values)

        # Return the dataset according to wordvecs' source
        return torch.tensor(data[self.source], dtype=torch.float32)

In [302]:
dataset_ae = AutoEncoderDataset(sample_10p, w2v_encoder, llm_encoder,
                                source="mix")

## Autoencoder training

In [304]:
class Autoencoder(BaseEstimator, TransformerMixin, nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim,
                 lr=1e-4, epochs=1, batch_size=256):

        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim),
            nn.Tanh()
        )

    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, X):
        return self.encoder(X)

    def fit(self, X):
        dataloader = DataLoader(X, batch_size=self.batch_size, shuffle=True)

        # Define optimizer and loss function
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.parameters(), lr=self.lr)

        # Training loop
        self.train()
        for epoch in range(self.epochs):
            losses = 0
            with tqdm(total=len(dataloader),
                      desc=f"Epoch {epoch + 1}", unit="batch") as pbar:

                for X_batch in dataloader:
                    reconstructed_X = self.forward(X_batch)
                    loss = criterion(reconstructed_X, X_batch)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    losses += loss.item()

                    pbar.set_postfix({'Loss': loss.item()})
                    pbar.update(1)

                print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {(losses/len(dataloader)):.3f}")

        return self

    def transform(self, X):
        self.eval()

        encoded = []

        dataloader = DataLoader(X, batch_size=self.batch_size, shuffle=False)
        for X_batch in dataloader:
            with torch.no_grad():
                encoded.append(self.encode(X_batch).cpu().numpy())

        return np.vstack(encoded)

In [370]:
# Define the hyperparameters for autoencoder training
ae_hyperparams = {
    "input_dim": dataset_ae.get_feature_dim(),
    "hidden_dim": 1024,
    "latent_dim": 410,
    "lr": 1e-5,
    "epochs": 1,
    "batch_size": 256
}

In [372]:
# Train autoencoder
model_ae = Autoencoder(**ae_hyperparams)
wv_ae = model_ae.fit(dataset_ae)

Epoch 1: 100%|██████████| 127/127 [13:16<00:00,  6.27s/batch, Loss=0.00109]

Epoch 1/1, Loss: 0.001


In [373]:
# Save model for future access
# torch.save(model_ae.state_dict(), f"autoencoder")

## Clustering

In [374]:
selected_data = select_dataset(processed_dataset, business_id, model_sentiment)

# Get the word vector from the trained autoencoder
selected_data_ae = AutoEncoderDataset(selected_data, w2v_encoder, llm_encoder,
                                      source="mix")
selected_data["w2v_ae"] = list(model_ae.transform(selected_data_ae))

# Initialize evaluation results
evaluation = []

# Initialize subplots
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=["Positive Reviews", "Negative Reviews"])

for i, sentiment in enumerate(["positive", "negative"], start=1):

    # Clustering
    clust = HDBSCANClustering(apply_pca=True)
    X = selected_data[selected_data["sentiment"] == sentiment].copy()
    X_ = np.vstack(X["w2v_ae"].values)
    X["w2v_ae_label"] = clust.fit_transform(X_)

    # Evaluation
    evaluation.append(evaluate_cluster(X_, X["w2v_ae_label"], sentiment))

    # Visualization
    visualize_cluster(fig, X, "w2v_ae", "w2v_ae_label",
                      col=i, sentiment=sentiment)

fig.update_layout(title="Customer Review Clusters - Positive and Negative")
fig.show()

In [375]:
# Evaluation
pd.concat(evaluation, axis=1)

,Positive Reviews Clustering,Negative Reviews Clustering
silhouette_score,0.051932,0.138052
davies_bouldin_score,2.033052,1.350578
calinski_harabasz_score,2.369022,2.165525


## Cluster labeling

In [ ]:
import openai

def get_cluster_name(text_samples):
    prompt = f"""
        You are an expert in giving a descriptive topic to a given list of sentences.
        The sentences may have different topics, so choose one that is the most commonly shared.
        Please return the topic as consice as possible, maximum in 3 words.
        Please also avoid a vague topic.

        There are 5 sentences as the input.
        The content of the sentences is limited to customer reviews for a tourist attraction.
        So, please only choose the topic according to possible aspects in this business.

        Here is an example with just 3 setences:

        INPUT
        'Gorgeous place to visit, it can get crowded on holidays.'
        'Great hike and beautiful views.'
        'Awesome view.'
        ENDINPUT

        LABEL 'Scenic view'

        So here is the sentences:
        INPUT
        {'\n'.join(text_samples)}
        ENDINPUT

        LABEL ...
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message['content'].strip()